In [1]:
%%sql
CREATE SHEMA resteurant_sch;
GO
# Ingridient
CREATE TABLE resteurant_sch.ingridient(
ID INT PRIMARY KEY,
name VARCHAR(64) NOT NULL,
price DECIMAL(10,2) NOT NULL,
isVegan BOOLEAN NOT NULL,
isVegeterian BOOLEAN NOT NULL
);

# Pizza ingredient
CREATE TABLE resteurant_sch.pizzaIngridient(
pizzaID INT NOT NULL,
ingridientID INT NOT NULL,
quantity INT NOT NULL
);

# Item
CREATE TABLE resteurant_sch.item(
ID INT PRIMARY KEY,
itemType VARCHAR(6) NOT NULL,
name VARCHAR(64) NOT NULL,
price DECIMAL(10,2) NOT NULL,
isVegan BOOLEAN,
isVegeterian BOOLEAN
);

# Order item
CREATE TABLE resteurant_sch.orderItem(
ID INT PRIMARY KEY,
itemID INT NOT NULL,
quantity INT NOT NULL
);

# Discount code
CREATE TABLE resteurant_sch.discountCode(
ID VARCHAR(8) PRIMARY KEY,
isUSED BOOLEAN NOT NULL,
percetage INT NOT NULL,
);

# Delivery personel
CREATE TABLE resteurant_sch.deliveryPersonel(
ID INT PRIMARY KEY,
firstName VARCHAR(64) NOT NULL,
lastName VARCHAR(64) NOT NULL,
postalCode VARCHAR(6) NOT NULL
);

# Customer
CREATE TABLE resteurant_sch.customer(
ID INT PRIMARY KEY,
firstName VARCHAR(64) NOT NULL,
lastName VARCHAR(64) NOT NULL,
gender CHAR(1) NOT NULL,
birthDate DATETIME NOT NULL,
phoneNumber VARCHAR(12) NOT NULL,
email VARCHAR(100) NOT NULL,
adress VARCHAR(100) NOT NULL,
postalCode VARCHAR(6) NOT NULL,
city VARCHAR(64) NOT NULL,
# how many pizzas did the customer order (for the discount after 10)
pizzaCount INT NOT NULL
);

# Order
CREATE TABLE resteurant_sch.order(
ID INT PRIMARY KEY,
customerID INT NOT NULL,
placementDate DATETIME NOT NULL,
pickedUpByDeliveryPersonelDate DATETIME,
status VARCHAR(32) NOT NULL,
price DECIMAL(10,2) NOT NULL,
discountCodeID VARCHAR(8),
deliveryPersonelID INT
);



UsageError: Cell magic `%%sql` not found.
